In [1]:
import ollama
import os
import pandas as pd

device = "RTX4060"
batch_sizes = [1,8,32,128]
outputs = []

In [4]:
# warmup
# context 32K
# Internet Access NO

def bench(prompts,prefix = "", stream = False):
    result = {}
    output = ollama.chat(
        model="llama3",
        messages=[
            {"role": "user", "content": prefix},
        ] + [{"role": "user", "content": prompt+"\n"} for prompt in prompts],
        stream = stream
    )

    result["prompts"] = len(prompts)
    result["input_tokens"] = output.prompt_eval_count
    result["output_tokens"] = output.eval_count
    result["total_time"] = output.total_duration/1e+6
    result["load_time"] = output.load_duration/1e+6
    result["prompt_eval_time"] = output.prompt_eval_duration/1e+6
    result["output_eval_time"] = output.eval_duration/1e+6

    return result, output.message.content

bench(["Do you know whos dashing, smart and disappears in a flash"])

({'prompts': 1,
  'input_tokens': 25,
  'output_tokens': 16,
  'total_time': 480.5932,
  'load_time': 129.3574,
  'prompt_eval_time': 12.0275,
  'output_eval_time': 330.5215},
 'I think I can solve the riddle!\n\nIs it "The Flash"?')

In [5]:
files = os.listdir("data/prompts/")
for filename in files:
    _, questions, prompt_complexity, prompt_length = filename.split(".")[0].split("_")

    for batch_size in batch_sizes:
        with open(f"./data/prompts/{filename}", "r") as file:
            prompts = file.read().split("\\n")[:batch_size]

            for i in range(len(prompts)):
                prompts[i] = f"Question {i+1} : {prompts[i].strip()}"

        for output_length in ["short", "long"]:
            if output_length == "short":
                benchmark, answer = bench(prompts, prefix = f"give short answers for each of the following {len(prompts)} questions")
            else:
                benchmark, answer = bench(prompts, prefix = f"give long answers for each of the following {len(prompts)} questions")

            with open(f"./data/answers/answer_{batch_size}_{prompt_complexity}_{prompt_length}.md", "w", encoding="utf-8") as file:
                file.write(answer)

            benchmark["prompt_complexity"] = prompt_complexity
            benchmark["prompt_length"] = prompt_length
            benchmark["output_length"] = output_length
            benchmark["device"] = device

            outputs.append(benchmark)
            outputs_df = pd.DataFrame(outputs)
            outputs_df.to_csv(f"./data/{device}_outputs.csv", index = False)

In [9]:
outputs_df = pd.read_csv(f"./data/{device}_outputs.csv")
outputs_df

,prompts,input_tokens,output_tokens,total_time,load_time,prompt_eval_time,output_eval_time,prompt_complexity,prompt_length,output_length,device
0,1,87,23,625.3683,116.8789,12.5857,490.9290,easy,long,short,RTX3060
1,1,87,439,9596.8888,115.8751,17.3173,9460.5983,easy,long,long,RTX3060
2,8,506,181,4175.7532,131.2558,47.9121,3983.5546,easy,long,short,RTX3060
3,8,506,1494,32957.6929,122.2153,49.1032,32783.4597,easy,long,long,RTX3060
4,32,1928,581,14087.9400,112.5312,851.0502,13079.4369,easy,long,short,RTX3060
5,32,1928,2101,49099.4195,117.6494,845.6461,48096.0429,easy,long,long,RTX3060
6,128,7646,617,23061.0483,120.0094,5541.1119,16922.1482,easy,long,short,RTX3060
7,128,7646,10373,285439.5960,132.9266,5528.9050,279218.7624,easy,long,long,RTX3060
8,1,33,2,180.1509,120.2273,16.0432,43.3757,easy,short,short,RTX3060
9,1,33,344,7387.5351,116.7438,11.4549,7258.8309,easy,short,long,RTX3060
